## Foil  
**该库用 Cursor 实现了知识图谱中的一阶规则学习算法，它遵循序贯覆盖框架，采用自顶向下的规则归纳策略。**  

**我对这个算法的理解就是纯搜索，然后剪枝。**  

**代码使用：**  

> python foil.py  

**输入**  

5  

Couple(James, David)  

Mother(James, Mike)  

Father(David, Mike)  

Mother(James, Ann)  

Sibling(Ann, Mike)  

Father  

**输出**  

Couple(z, x), Mother(z, y) -> Father(x, y)  

**输入**  

18  

succ(0,1)  

succ(1,2)  

succ(2,3)  

succ(3,4)  

succ(4,5)  

succ(5,6)  

succ(6,7)  

succ(7,8)  

succ(8,9)  

pred(1,0)  

pred(2,1)  

pred(3,2)  

pred(4,3)  

pred(5,4)  

pred(6,5)  

pred(7,6)  

pred(8,7)  

pred(9,8)  

pred  

输出  

succ(y, x) -> pred(x, y)  


In [10]:
# 导入数学库
import math
from collections import defaultdict

Fact 类的作用  

* 封装逻辑谓词和参数（如 Parent(John, Mary)）  
* 支持字符串表示、相等性判断和哈希操作，用于后续规则匹配和数据集管理  
* 是 FOIL 算法中存储正例/反例、规则头/规则体的基础单元

In [2]:
# 定义事实类（用于表示逻辑谓词事实，例如 Parent(John, Mary)）
class Fact:
    def __init__(self, predicate, arguments):
        """
        初始化事实对象
        :param predicate: 谓词名称（如 "Parent"）
        :param arguments: 谓词参数列表（如 ["John", "Mary"]）
        """
        self.predicate = predicate
        self.arguments = arguments
    
    def __str__(self):
        """将事实转换为字符串（如 Parent(John, Mary)）"""
        args_str = ", ".join(self.arguments)
        return f"{self.predicate}({args_str})"
    
    def __eq__(self, other):
        """判断两个事实是否相同（用于规则匹配和集合操作）"""
        return (self.predicate == other.predicate and 
                self.arguments == other.arguments)
    
    def __hash__(self):
        """生成哈希值（用于存储在哈希集合或字典中）"""
        return hash((self.predicate, tuple(self.arguments)))

In [3]:
# 验证 Fact 类的功能
def test_fact():
    # 输入测试
    fact1 = Fact("Parent", ["John", "Mary"])
    fact2 = Fact("Parent", ["John", "Mary"])
    fact3 = Fact("Sibling", ["Alice", "Bob"])
    
    # 输出验证
    assert str(fact1) == "Parent(John, Mary)", "字符串表示错误"
    assert fact1 == fact2, "相同事实应判断为相等"
    assert fact1 != fact3, "不同事实应判断为不等"
    assert hash(fact1) == hash(fact2), "相同事实的哈希值应一致"
    
    print("所有测试通过！")

test_fact()

所有测试通过！


Rule 类的作用  

* 表示逻辑规则（如 A ← B ∧ C），用于 FOIL 算法中的规则生成与推理  
* head: 规则的目标谓词（如 Grandparent(X,Z)）  
* body: 前提条件列表（如 [Parent(X,Y), Parent(Y,Z)]）  
* 支持灵活构造规则（允许规则体为空，表示无条件成立）

In [4]:
# 定义规则类（表示逻辑规则，如 Grandparent(X,Z) ← Parent(X,Y) ∧ Parent(Y,Z)）
class Rule:
    def __init__(self, head, body=None):
        """
        初始化规则对象
        :param head: 规则头部（Fact 类实例，如 Grandparent(X,Z)）
        :param body: 规则体（Fact 类实例列表，如 [Parent(X,Y), Parent(Y,Z)]）
        """
        self.head = head
        self.body = body if body is not None else []  # 默认空列表
    
    def __str__(self):
        """将规则转换为字符串（如 Parent(X,Y), Parent(Y,Z) -> Grandparent(X,Z)）"""
        if not self.body:
            return str(self.head)  # 无规则体时直接返回头部
        body_str = ", ".join(str(lit) for lit in self.body)
        return f"{body_str} -> {self.head}"

In [5]:
# 验证 Rule 类的功能
def test_rule():
    # 输入测试
    parent_fact = Fact("Parent", ["X", "Y"])
    grandparent_fact = Fact("Grandparent", ["X", "Z"])
    rule = Rule(grandparent_fact, [parent_fact, Fact("Parent", ["Y", "Z"])])

    # 输出验证
    expected_str = "Parent(X, Y), Parent(Y, Z) -> Grandparent(X, Z)"
    assert str(rule) == expected_str, f"字符串表示错误，实际值: {str(rule)}"
    
    # 测试空规则体
    simple_rule = Rule(parent_fact)
    assert str(simple_rule) == "Parent(X, Y)", "空规则体处理错误"
    
    print("所有测试通过！")

test_rule()

所有测试通过！


$$  
 FOIL\_Gain = \widehat{m}_+ \cdot \left( \log_2 \frac{\widehat{m}_+}{\widehat{m}_+ + \widehat{m}_-} - \log_2 \frac{m_+}{m_+ + m_-} \right)  
$$  
符号说明  
$m_+$: 添加新文字前覆盖的正例数  
$m_-$: 添加新文字前覆盖的负例数  
$\hat{m}_+$ : 添加新文字后覆盖的正例数  
$\hat{m}_-$ : 添加新文字后覆盖的负例数

In [11]:
class FOILAlgorithm:
    def __init__(self):
        """初始化 FOIL 算法核心数据结构"""
        self.facts = []                # 存储所有已知事实
        self.constants = set()         # 常量集合（如实体名称）
        self.predicates = defaultdict(list)  # 谓词到参数列表的映射
        self.target_predicate = ""     # 目标谓词（要学习的规则头）
        
    def add_fact(self, fact_str):
        """
        解析并存储事实字符串（如 "Parent(James, Mike)"）
        :param fact_str: 符合语法的谓词字符串
        """
        predicate, args_str = fact_str.strip().split("(")
        args_str = args_str.rstrip(")")
        arguments = [arg.strip() for arg in args_str.split(",")]
        
        fact = Fact(predicate, arguments)
        self.facts.append(fact)
        
        # 更新常量和谓词库
        self.constants.update(arguments)
        self.predicates[predicate].append(arguments)
    
    def set_target_predicate(self, predicate):
        """设置目标谓词（规则学习的目标）"""
        self.target_predicate = predicate
    
    def generate_all_possible_facts(self, predicate, arity):
        """
        生成指定谓词的所有可能参数组合
        :param predicate: 谓词名称
        :param arity: 参数元数（如二元关系）
        :return: 所有可能的事实列表
        """
        import itertools
        return [Fact(predicate, list(args)) 
                for args in itertools.product(self.constants, repeat=arity)]
    
    def get_positive_examples(self):
        """获取目标谓词的已知正例"""
        return [Fact(self.target_predicate, args) 
                for args in self.predicates.get(self.target_predicate, [])]
    
    def get_negative_examples(self):
        """生成目标谓词的负例（全集-正例）"""
        positive = self.get_positive_examples()
        
        # 推断目标谓词的元数
        arity = len(positive[0].arguments) if positive else 2
        
        all_possible = self.generate_all_possible_facts(self.target_predicate, arity)
        return [fact for fact in all_possible if fact not in positive]
    
    def evaluate_literal(self, literal, bindings):
        """
        评估给定绑定下文字是否为真
        :param literal: 要验证的 Fact 对象
        :param bindings: 变量到常量的映射（如 {'?x': 'Alice'}）
        :return: 是否满足条件
        """
        if literal.predicate not in self.predicates:
            return False
        
        # 替换变量为绑定值
        grounded_args = []
        for arg in literal.arguments:
            grounded_args.append(bindings.get(arg, arg))
        
        return grounded_args in self.predicates[literal.predicate]
    
    def covers(self, rule, example, bindings=None):
        """
        判断规则是否覆盖给定例子
        :param rule: 规则对象
        :param example: 目标谓词的实例
        :param bindings: 变量绑定（递归验证时传入）
        :return: 是否覆盖
        """
        if bindings is None:
            # 初始化头部变量绑定
            bindings = {}
            for head_arg, ex_arg in zip(rule.head.arguments, example.arguments):
                if head_arg.startswith('?'):
                    bindings[head_arg] = ex_arg
        
        # 验证所有规则体文字
        for literal in rule.body:
            matched = False
            # 遍历所有可能的参数组合
            for args in self.predicates.get(literal.predicate, []):
                new_bindings = bindings.copy()
                valid = True
                
                # 匹配变量与常量
                for lit_arg, arg in zip(literal.arguments, args):
                    if lit_arg.startswith('?'):
                        if lit_arg in new_bindings:
                            if new_bindings[lit_arg] != arg:
                                valid = False
                                break
                        else:
                            new_bindings[lit_arg] = arg
                    elif lit_arg != arg:
                        valid = False
                        break
                
                if valid:
                    bindings.update(new_bindings)
                    matched = True
                    break
            
            if not matched:
                return False
        
        return True
    
    def foil_gain(self, rule, new_literal, pos, neg):
        """
        计算添加新文字的信息增益
        :param rule: 当前规则
        :param new_literal: 候选新文字
        :param pos: 正例列表
        :param neg: 负例列表
        :return: FOIL 信息增益值
        """
        p0, n0 = len(pos), len(neg)
        if p0 == 0:
            return 0
        
        # 构建新规则并计算覆盖情况
        new_rule = Rule(rule.head, rule.body + [new_literal])
        p1 = sum(1 for ex in pos if self.covers(new_rule, ex))
        n1 = sum(1 for ex in neg if self.covers(new_rule, ex))
        
        if p1 == 0:
            return 0
        
        # FOIL 增益公式实现
        return p1 * (math.log2(p1/(p1+n1)) - math.log2(p0/(p0+n0)))
    
    def learn_rule(self):
        """核心规则学习算法"""
        pos = self.get_positive_examples()
        neg = self.get_negative_examples()
        
        if not pos:
            return None
        
        # 初始化规则（假设二元目标谓词）
        head = Fact(self.target_predicate, ["?x", "?y"])
        rule = Rule(head)
        
        # 迭代添加文字直到不覆盖负例
        while neg and any(self.covers(rule, ex) for ex in neg):
            best_literal = None
            best_gain = -1
            
            # 遍历所有可能的候选文字
            for pred in self.predicates:
                if pred == self.target_predicate:
                    continue
                
                # 获取谓词元数
                arity = len(self.predicates[pred][0]) if self.predicates[pred] else 1
                
                # 收集当前规则中的变量
                variables = set()
                for lit in [rule.head] + rule.body:
                    variables.update([arg for arg in lit.arguments if arg.startswith('?')])
                
                # 生成所有可能的参数组合（允许引入新变量 ?z）
                import itertools
                for args in itertools.product(list(variables) + ["?z"], repeat=arity):
                    literal = Fact(pred, list(args))
                    gain = self.foil_gain(rule, literal, pos, neg)
                    
                    if gain > best_gain:
                        best_gain = gain
                        best_literal = literal
            
            if not best_literal or best_gain <= 0:
                break  # 无法进一步优化
            
            # 更新规则并筛选例子
            rule.body.append(best_literal)
            pos = [ex for ex in pos if self.covers(rule, ex)]
            neg = [ex for ex in neg if self.covers(rule, ex)]
        
        return rule
    
    def run(self):
        """执行算法并返回可读规则"""
        rule = self.learn_rule()
        if not rule:
            return "未找到规则"
        
        # 重命名变量为易读形式（x, y, z, v0, v1...）
        var_map = {}
        var_count = 0
        for lit in [rule.head] + rule.body:
            for arg in lit.arguments:
                if arg.startswith('?') and arg not in var_map:
                    var_map[arg] = ['x', 'y', 'z'][var_count] if var_count < 3 else f'v{var_count-3}'
                    var_count += 1
        
        # 替换变量名
        for lit in [rule.head] + rule.body:
            for i in range(len(lit.arguments)):
                if lit.arguments[i] in var_map:
                    lit.arguments[i] = var_map[lit.arguments[i]]
        
        return str(rule)

**按行读取**  
输入你需要输入的规则数量  
例如：5  
然后输入规则  
例如  
Couple(James,David)  
Mother(James,Mike)  
Father(David,Mike)  
Mother(James,Ann)  
Sibling(Ann,Mike)  
最后输入你需要推导的规则  
例如  
Father

In [13]:
#Test 1
# 读取输入
n = int(input().strip())

foil = FOILAlgorithm()

# 读取已有关系
for _ in range(n):
    fact_str = input().strip()
    foil.add_fact(fact_str)

# 读取目标谓词
target_predicate = input().strip()
foil.set_target_predicate(target_predicate)

# 运行FOIL算法并输出结果
result = foil.run()
print(result)

5

Couple(James,David)

Mother(James,Mike)

Father(David,Mike)

Mother(James,Ann)

Sibling(Ann,Mike)

Father

Couple(z, x), Mother(z, y) -> Father(x, y)
